In [2]:
# https://www.kaggle.com/kernels/scriptcontent/11511967/notebook

import string
import numpy as np
import pandas as pd
# from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast
import re
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
# from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

import nltk
from nltk.tag import tnt
from nltk.corpus import stopwords
from nltk.corpus import indian
from nltk.tokenize import word_tokenize,sent_tokenize

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora
from gensim.summarization import summarize
from gensim.test.utils import datapath

import stanfordnlp
import networkx as nx
import math

import warnings
warnings.filterwarnings("ignore")

# from bokeh.plotting import figure, output_file, show
# from bokeh.models import Label
# from bokeh.io import output_notebook
# output_notebook()

%matplotlib inline

/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, an

In [2]:
nlp = stanfordnlp.Pipeline ( lang = 'hi' ) 

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_tokenizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_tagger.pt', 'pretrain_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb.pretrain.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_lemmatizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_parser.pt', 'pretrain_path': '/Users/rohanraj/stanford

In [3]:
datafile = '../data/hindi.txt'
with open(datafile,'r',encoding='utf-8') as f:
    text = f.read()
    text = text.split("॥")

In [179]:
text_sentence = []
i = 0
for word in tqdm(text):
#     print(i,word)
    if re.findall("^\s?\d+\s?",word):
        continue
    else:   
        text_sentence.append(word)

100%|██████████| 1290/1290 [00:00<00:00, 427145.51it/s]


In [180]:
text_sentence[0:2]

[' वानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास्त की प्रतीक्षा करता भया ',
 ' घ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया ']

In [6]:
len(text_sentence) #total sentences

667

In [7]:
# hindi stopwords
stop_words_df = pd.read_csv("../data/stopwords.txt", header = None)
stop_words = list(set(stop_words_df.values.reshape(1,-1).tolist()[0]))
stop_words.extend(["।", "।।", ")", "(", ",",'"',"हे", "हो", 'में','से','COMMA'])

In [8]:
len(stop_words)

723

In [10]:
stop_words[:2]

['सकती', 'बारे']

In [11]:
def create_str_from_list(original_read_text):
    prepared_text = ""
    for line in original_read_text:
        line = line.split()
#       print(line)
        tmp_line = " ".join(line)
        prepared_text += " \n"+tmp_line
    return prepared_text

In [12]:
str_text = create_str_from_list(text_sentence)

In [13]:
str_text[0:2500]

' \nवानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास्त की प्रतीक्षा करता भया \nघ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया \nधानरराज के उस हितैषी ने लंका नगरी में प्रवेश करके मानों अपना बायां पाओं शत्रु के सिर पर रख दिया \nउस समय सुन्दर \'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ \nएक भवन से दूसरे भवन को जाते हुए उस वानरश्रेष्ठ ने वहाँ र विविध भाकृति और रूपों वाले भवन देखे \nवहाँ राक्षसों के घरों में उसने अप "करते हुओं के मन्त्र मुने और स्वाध्याय में रत राक्षसों को देखा \nराक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा \nतव पवनपुत्र हनुमान ने यह भवन श्रेष्ठ देखा, जो राक्षसपति का भवन है, और बहुत महलों से भरपूर है \nविशाल नेत्रोंवाली वैदेही सीता को ढूंढता हुआ शत्रुनों के मारनेवाला, हनुमान उसके चारों ओर घूमा ।। ९ \nतब 

In [14]:
summarize(str_text[0:2500])

"घ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया \nउस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन.\nराक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा \nतब वह उस सुन्दर पड़ी माला की ओर मस्थित हुआ, जो उत्तम स्त्री की तरह रावण की बड़ी पारी थी \nमें बस वानर यूयपति ने चन्द्र तुल्य मुखवाली, सुन्दर कुण्डल पहने हुई, ताजे घुषषों की पालाएं और भूषणोंवाली पनियों को देसा "

In [19]:
len(str_text)

68718

In [17]:
df = pd.DataFrame()

In [67]:
def summarization(str_text):
    startflag = 0
    endflag = 2500
    parsed_text = {'text':[], 'title':[]}
    for summary in range(int(len(str_text)/endflag)):
        summarization_block = summarize(str_text[startflag:endflag])
        parsed_text['text'].append(str_text[startflag:endflag])
        parsed_text['title'].append(summarization_block)
        startflag +=2500
        endflag +=2500
    return pd.DataFrame(parsed_text)

In [68]:
df = summarization(str_text)
df.shape

(27, 2)

In [69]:
df.to_csv('input_abstarct_summary.csv')

### Alternative Summarization from Gensim 

In [220]:
def textrankTfIdf(document):
    # sentence_tokenizer = PunktSentenceTokenizer()
    # sentences = sentence_tokenizer.tokenize(document, 'hindi')

    sentences = document
    bow_matrix = CountVectorizer().fit_transform(sentences)
    # normalized = TfidfTransformer(norm='l2', use_idf=True, use_bm25idf=True, smooth_idf=True,
    #              delta_idf=False, sublinear_tf=False, bm25_tf=True).fit_transform(bow_matrix)

    normalized = TfidfTransformer().fit_transform(bow_matrix)
    similarity_graph = normalized * normalized.T

    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i], s) for i, s in enumerate(sentences)),
                  reverse=True)
def orderSentences(rankedList, data, initSentences):
    index = ['']*len(data)
    # print(rankedList)
    for eachRanked in rankedList[0:int(math.ceil(0.2*len(rankedList)))]:
        sen = eachRanked[1]
        index[data.index(sen)] = initSentences[data.index(sen)]
        # print(data.index(sen))
    return index

In [221]:
int(len(text_sentence)/25)

26

In [222]:
def alternate_summarization(text_sentence):
    startflag = 0
    endflag = 25
    parsed_text = {'text':[], 'alt_title':[]}
    sentence =''
    for summary in range(int(len(text_sentence)/endflag)):
        rankedSentences = textrankTfIdf(text_sentence[startflag:endflag])
        orderedsentences = orderSentences(rankedSentences, text_sentence[startflag:endflag], text_sentence[startflag:endflag])
        for ordered in orderedsentences:
            if ordered != "":
                sentence += ordered 
        parsed_text['alt_title'].append(sentence)
        parsed_text['text'].append(text_sentence[startflag:endflag])
        startflag +=25
        endflag +=25
        sentence =''
    return pd.DataFrame(parsed_text)


In [223]:
df2 = alternate_summarization(text_sentence)

In [224]:
df2.head()

,text,alt_title
0,[ वानर हनुमान सीमा के देखने की उत्कण्ठा येसूया...,उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के ...
1,[ मेरी दृष्टि आजतक (ऐमी अवस्था में ) परस्त्रिय...,सो मैंने शुद्ध मन से रावण का पारा अन्तःपुर ढू...
2,[ तब उस वानर ने सारे जगह में सुहावना एक पर्वत ...,तब उस वानर ने सारे जगह में सुहावना एक पर्वत द...
3,"[ , १८) प्रभात का समय और रावण का अशोकवनिका में...",उस समय मङ्गक बाओं और कानों के प्यारे शब्दों स...
4,"[ पह तेरा सुन्दर बना हुथा, यौवन चला जारहा है, ...",सारे लोकों से पल से हर कर जो मैं रत्न लाचा हू...


In [226]:
df2.to_csv('summary_alt_gensim.csv')

In [4]:
df2 = pd.read_csv('../data/summary_alt_gensim.csv')

In [69]:
def extract_ner(sentence_ner):
    parsed_text = {'word':[], 'ner':[]}
    words = re.findall("([^<]+)<[^>]+>",sentence_ner)
    tags = re.findall("<([^>]+)>",sentence_ner)
    
    # for word in words: append word #we need to write seperate tags
    
    for index in range(len(words)):
        parsed_text['word'].append(words[index])
        parsed_text['ner'].append(tags[index])
    return pd.DataFrame(parsed_text)

In [70]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the model you trained
model = SequenceTagger.load('/Users/rohanraj/Documents/Master/CaseStudy2/code/resources/taggers/netag/best-model.pt')

df_ner = pd.DataFrame()

r, c = df2.shape
for row_no in range(r):
    sentence = Sentence(df2['alt_title'][row_no])
    model.predict(sentence)
    sentence_ner = sentence.to_tagged_string()
    temp_df = extract_ner(sentence_ner)
    df_ner = df_ner.append(temp_df,ignore_index=True)  
# predict tags and print
#print(sentence.to_tagged_string())


2020-03-03 19:40:57,907 loading file /Users/rohanraj/Documents/Master/CaseStudy2/code/resources/taggers/netag/best-model.pt


In [73]:
df_ner.to_csv('../data/ner_tag.csv')

In [48]:
sentence_ner = sentence.to_tagged_string()

In [56]:
y= re.findall("([^<]+)<[^>]+>",sentence_ner)

In [51]:
x = re.findall("([^<]+)<[^>]+>",sentence_ner)

In [53]:
re.findall("<([^>]+)>",sentence_ner)

['o',
 'o',
 'B-PERSON',
 'B-LOCATION',
 'B-PERSON',
 'o',
 'o',
 'o',
 'B-PERSON',
 'B-PERSON',
 'o',
 'o',
 'B-PERSON',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'B-PERSON',
 'o',
 'o',
 'o',
 'B-PERSON',
 'o',
 'o',
 'o',
 'B-PERSON',
 'o',
 'B-PERSON',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'B-COUNT',
 'B-COUNT',
 'o',
 'B-PERSON',
 'B-PERSON',
 'o',
 'o',
 'o',
 'B-PERSON',
 'o',
 'o',
 'B-PERSON',
 'o',
 'B-PERSON',
 'B-PERSON',
 'B-PERSON',
 'o',
 'B-PERSON',
 'B-PERSON',
 'o',
 'o',
 'o',
 'B-PERSON',
 'B-LOCATION',
 'B-PERSON',
 'o',
 'I-LOCATION',
 'I-LOCATION',
 'o',
 'B-PERSON',
 'o',
 'B-PERSON',
 'B-LOCATION',
 'B-PERSON',
 'B-PERSON',
 'B-LOCATION',
 'B-PERSON',
 'B-PERSON',
 'B-LOCATION',
 'B-PERSON',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'B-PERSON',
 'o',
 'B-PERSON',
 'B-LOCATION',
 'B-PERSON',
 'o',
 'B-PERSON',
 'B-PERSON',
 'B-PERSON',
 'B-PERSON',
 'B-PERSON',
 'B-PERSON',
 'B-PERSON',
 'o',
 'B-PERSON',
 'B-PERSON',
 'B-PERSON',


In [9]:
#re.findall("<[A-Z]-[A-Z]*>",sentence)

['<B-PERSON>',
 '<B-LOCATION>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-COUNT>',
 '<B-COUNT>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-LOCATION>',
 '<B-PERSON>',
 '<I-LOCATION>',
 '<I-LOCATION>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-LOCATION>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-LOCATION>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-LOCATION>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-LOCATION>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<I-LOCATION>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-LOCATION>',
 '<B-PERSON>',
 '<B-PERSON>',
 '<B-PERSON>']

In [65]:
extract_ner_df = extract_ner(sentence)
extract_ner_df.head()

,word,ner
0,उस,o
1,समय,o
2,सुन्दर,B-PERSON
3,'सब,B-LOCATION
4,ओर,B-PERSON


In [231]:
print(sentence.tokens)

[Token: 1 उस, Token: 2 समय, Token: 3 सुन्दर, Token: 4 'सब, Token: 5 ओर, Token: 6 से, Token: 7 सजे, Token: 8 हुए, Token: 9 श्वेत, Token: 10 मेघ, Token: 11 के, Token: 12 तुल्य, Token: 13 राक्षसों, Token: 14 के, Token: 15 जो, Token: 16 पद्माकार,, Token: 17 स्वस्तिकाकार,, Token: 18 और, Token: 19 वर्धमान, Token: 20 घर, Token: 21 है,, Token: 22 उन, Token: 23 से, Token: 24 इंका, Token: 25 जगमन., Token: 26 कर, Token: 27 रही, Token: 28 थी,, Token: 29 राधा, Token: 30 के, Token: 31 अर्थ, Token: 32 वह, Token: 33 श्रीमान्, Token: 34 घूमता, Token: 35 हुमा, Token: 36 उसे, Token: 37 देखता, Token: 38 भया, Token: 39 और, Token: 40 आनन्दित, Token: 41 होता, Token: 42 भया, Token: 43 ।।, Token: 44 ३,, Token: 45 ४, Token: 46 वहाँ, Token: 47 राक्षसों, Token: 48 के, Token: 49 घरों, Token: 50 में, Token: 51 उसने, Token: 52 अप, Token: 53 "करते, Token: 54 हुओं, Token: 55 के, Token: 56 मन्त्र, Token: 57 मुने, Token: 58 और, Token: 59 स्वाध्याय, Token: 60 में, Token: 61 रत, Token: 62 राक्षसों, Token: 63 को, Token: 64

In [165]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the model you trained
model = SequenceTagger.load('/Users/rohanraj/Documents/Master/CaseStudy2/code/resources/taggers/netag/best-model.pt')

# create example sentence
#sentence = Sentence('मोहनदास करमचन्द गांधी (२ अक्टूबर १८६९ - ३० जनवरी १९४८) भारत एवं भारतीय स्वतंत्रता आंदोलन के एक प्रमुख राजनैतिक एवं आध्यात्मिक नेता थे।')
sentence = Sentence(df['title'][0])
# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

2020-03-01 20:29:04,306 loading file /Users/rohanraj/Documents/Master/CaseStudy2/code/resources/taggers/netag/best-model.pt
घ४-९) <o> सीता <B-PERSON> को <B-PERSON> रावण <B-PERSON> के <B-PERSON> भन्तापुर <B-LOCATION> में <B-PERSON> ढूंढना <o> : <o> वह <B-PERSON> महावीर्य <B-PERSON> महान <B-PERSON> हृदय <B-PERSON> पाछा <B-PERSON> वानर <B-PERSON> श्रेष्ठ <B-PERSON> रात <B-PERSON> के <o> समय <o> अद्वार <B-PERSON> से <o> कोट <B-PERSON> को <o> फांद <o> कर <o> का <o> में <o> प्रविष्ट <o> हृया <o> 
उस <o> समय <o> सुन्दर <o> 'सब <B-PERSON> ओर <o> से <o> सजे <o> हुए <o> श्वेत <B-PERSON> मेघ <o> के <o> तुल्य <o> राक्षसों <o> के <o> जो <o> पद्माकार, <o> स्वस्तिकाकार, <o> और <o> वर्धमान <o> घर <o> है, <o> उन <o> से <o> इंका <o> जगमन.
राक्षसों <B-PERSON> के <o> घर <o> से <o> घर <o> और <o> बगीचों <I-LOCATION> को <o> देखता <o> हुआ <o> बेधड़क <I-LOCATION> यह <o> महलों <I-LOCATION> के <o> पास <o> घूमा <o> 
तब <o> वह <o> उस <o> सुन्दर <B-PERSON> पड़ी <o> माला <I-LOCATION> की <I-LOCATION> ओर <o> मस्थित <B

In [73]:
# Standford nlp is being used for lemmatization

In [95]:
def lemmatization(text):
    lemmatized_text = []
    for line in tqdm(text):
        if line not in [""," "] :
            doc = nlp(line)
            for sent in doc.sentences:
                for wrd in sent.words:
                    #extract text and lemma
                    lemmatized_text.append(wrd.lemma)
    return lemmatized_text

def remove_stopwords(word_tokenized,stop_words):
    return [word for word in word_tokenized if word not in stop_words]

def custom_remove_garbage(original_words_list,list_of_garbage_words):
    tmp_list = [word for word in original_words_list if word not in list_of_garbage_words] # garbage list
    tmp_list = [word for word in tmp_list if len(re.findall("\d+",word))==0] # english numbers
    tmp_list = [word for word in tmp_list if len(re.findall("[a-zA-Z]+",word))==0] # english alphabets
    return tmp_list

def Diff(li1, li2): 
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
    return li_dif 

## Topic Meddling after prepocessing

In [149]:
def topic_modelling(text_sentence,stop_words):
    df = pd.DataFrame()
    startflag = 0
    endflag = 25
    for text in range(int(len(text_sentence)/endflag)):
        
        #Clean text after lemmatization
        lemmatized = lemmatization(text_sentence[startflag:endflag])
        clean_text = remove_stopwords(lemmatized,stop_words)
        #len(clean_text),len(lemmatized) #281,612
        
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in clean_text]
        #print(stripped[:25])
        #print(len(clean_text),len(stripped)) #281

        resultant = Diff(clean_text,stripped)
        #len(set(resultant)) #19

        final_text = custom_remove_garbage(stripped,resultant)
        #len(final_text) #265
        
        # Create Dictionary
        id2word = corpora.Dictionary([final_text])

        # Create Corpus
        texts = final_text

        # Term Document Frequency
        corpus = [id2word.doc2bow(texts)] 
        
        # Build LDA model

        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=10,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)
        #pprint(lda_model.print_topics())
        temp_file = datapath("model")
        lda_model.save(temp_file)
        lda = gensim.models.ldamodel.LdaModel.load(temp_file)
        lda.update(corpus)
        lda.save('model')
        temp_df = pd.DataFrame(lda.print_topics())
        df = df.append(temp_df,ignore_index=True) 
        startflag +=25
        endflag +=25
    return df

In [150]:
df4 = topic_modelling(text_sentence,stop_words)

100%|██████████| 25/25 [00:02<00:00,  9.37it/s]


In [151]:
df4.shape

(130, 2)

In [152]:
df4.tail(5)

,0,1
125,0,"0.005*""सेना"" + 0.005*""सारण"" + 0.005*""कह"" + 0.0..."
126,1,"0.005*""सेना"" + 0.005*""सारण"" + 0.005*""कह"" + 0.0..."
127,2,"0.005*""सेना"" + 0.005*""सारण"" + 0.005*""कह"" + 0.0..."
128,3,"0.005*""सेना"" + 0.005*""कह"" + 0.005*""सारण"" + 0.0..."
129,4,"0.036*""सेना"" + 0.029*""सारण"" + 0.025*""कह"" + 0.0..."


In [153]:
df4.to_csv('combined_model_topic.csv')

In [160]:
temp_file = datapath("model")
lda = gensim.models.ldamodel.LdaModel.load(temp_file)